In [51]:
%pip install scsu
%pip install construct

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [52]:
from construct import *
import construct
import os
import itertools
import sys
import scsu # support for SCSU charset

In [53]:
checked_uid_struct = Struct(
    "uid" / Array(3, Int32ul),
    "checksum" / Int32ul
)

store_header_struct = Struct(
    "iBackup" / Int32ul,
    "iHandle" / Int32sl,
    "iRef" / Int32sl,
    "iCrc" / Int16ul,
)

toc_header_struct = Struct(
    "primary" / Int32ul,
    "avail" / Int32sl,
    "count" / Int32ul
)

toc_delta_header_struct = Struct(
    "tocoff" / Int32ul,
    "iMagic" / Int16ul,
    "n" / Int8ul
)

toc_entry_struct = Struct(
    "handle" / Int8ul,
    "ref" / Int32ul
)

toc_delta_entry_struct = Struct(
    "handle" / Int32ul,
    "ref" / Int32ul
)

In [54]:
class TCardinalityImpl(Construct):
    def _parse(self, stream, context, path):
        n = stream_read(stream, 1, path)[0]
        if (n & 0x1) == 0:
            return n >> 1
        elif (n & 0x2) == 0:
            n |= stream_read(stream, 1, path)[0] << 8
            return n >> 2
        elif (n & 0x4) == 0:
            arr = stream_read(stream, 3, path)
            n |= arr[0] << 8
            n |= arr[1] << 16
            n |= arr[2] << 24
            return n >> 3
        else:
            raise ValueError("invalid TCardinality value")

    def _build(self, obj, stream, context, path):
        if not isinstance(obj, int):
            raise IntegerError(f"value {obj} is not an integer", path=path)
        if obj < 0:
            raise IntegerError(f"TCardinality cannot build from negative number {obj}", path=path)
        n = obj
        if n < 0x80:
            n <<= 1
            stream_write(stream, bytes([n]), 1, path)
        elif n < 0x4000:
            n <<= 2
            stream_write(stream, bytes([n & 0xFF, (n >> 8) & 0xFF]), 2, path)
        elif n < 0x20000000:
            n <<= 3
            stream_write(stream, bytes([n & 0xFF, (n >> 8) & 0xFF, (n >> 16) & 0xFF, (n >> 24) & 0xFF]), 4, path)
        else:
            raise IntegerError(f"value {obj} is out of range for TCardinality", path=path)
        return obj

TCardinality = TCardinalityImpl()

class StringSizeAdapter(Adapter):
    def _decode(self, obj, context, path):
        return obj // 2

    def _encode(self, obj, context, path):
        return obj * 2

TDbName = PascalString(StringSizeAdapter(TCardinality), "SCSU")

class ReadBitSequence(Construct):
    def _parse(self, stream, context, path):
        cxroot = context._root

        if '_read_bit_entry' not in cxroot:
            cxroot['_read_bit_entry'] = 0

            # print(f"no _read_bit_entry")
        
        cxroot['_read_bit_entry'] >>= 1
        if (cxroot['_read_bit_entry'] & 0x1000000) == 0:
            cxroot['_read_bit_entry'] = stream_read(stream, 1, path)[0] | 0xFF000000

            # print(f"pull {cxroot['_read_bit_entry']}")

        return cxroot['_read_bit_entry'] & 1

    def _build(self, obj, stream, context, path):
        raise ValueError("_build not supported for ReadBitSequence")


In [55]:
column_schema_struct = Struct(
    "name" / TDbName,
    "type" / Int8ul,
    "attributes" / Int8ul,
    "maxLength" / If((this.type >= 11) & (this.type <= 13), Int8ul)
)

key_col_def_struct = Struct(
    "name" / TDbName,
    "iLength" / Int8ul,
    "iOrder" / Int8ul
)

index_def_struct = Struct(
    "name" / TDbName,
    "comparison" / Int8ul,
    "isUnique" / Int8ul,
    "keys" / PrefixedArray(TCardinality, key_col_def_struct),
    "iTokenId" / Int32ul,
)

table_schema_struct = Struct(
    "name" / TDbName,
    "columns" / PrefixedArray(TCardinality, column_schema_struct),
    "cluster" / TCardinality,
    "iTokenId" / Int32ul,
    "indexes" / PrefixedArray(TCardinality, index_def_struct)
)

db_schema_struct = Struct(
    "uid" / Int32ul,
    "iVersion" / Int8ul,
    "iToken" / Int32ul,
    "tables" / PrefixedArray(TCardinality, table_schema_struct)
)

ATTRIB_NOT_NULL = 1

In [56]:
table_token_struct = Struct(
    "iHead" / Int32ul, # head cluster ID
    "iNext" / Int32ul, # next record id?
    "iCount" / TCardinality,
    "iAutoIncrement" / Int32ul
)

cluster_struct = Struct(
    "iNext" / Int32ul, # next record id?
    "iMembership" / BitsSwapped(Bitwise(Array(16, Bit))),
    "sizes" / Array(16, If(lambda this: this.iMembership[this._index], TCardinality)),
    "data" / Array(16, If(lambda this: this.sizes[this._index] is not None, Bytes(lambda this: this.sizes[this._index])))
    # "iMembership" / Int16ul
)

In [57]:
with open("./F901iC/Java/FJJAM.DB", "rb") as f:
    checked_uid = checked_uid_struct.parse(f.read(16))
    store_header = store_header_struct.parse(f.read(16))
    print(store_header)

    if store_header.iBackup & 1:
        raise ValueError("store is dirty!")

    data = bytearray()
    while True:
        buf = f.read(0x4000)
        if len(buf) == 0:
            break

        data.extend(buf)

        f.read(2) # skip frame descriptors, even though they're needed to get record sizes


Container: 
    iBackup = 233736
    iHandle = 0
    iRef = 118248
    iCrc = 41251


In [58]:
class StoreToc:
    def __init__(self):
        self.primary = 0
        self.entries = []

    def parse(self, data, offset):
        toc_header = toc_header_struct.parse(data[offset-12:offset])
        if toc_header.primary & 0x80000000:
            toc_delta_header = toc_delta_header_struct.parse(data[offset:offset+7])

            self.parse(data, toc_delta_header.tocoff)

            toc_delta_entries = Array(toc_delta_header.n, toc_delta_entry_struct).parse(data[offset+7:offset+7+(toc_delta_header.n*8)])

            self.entries.extend(itertools.repeat(-1, toc_header.count - len(self.entries)))
            for entry in toc_delta_entries:
                self.entries[(entry.handle & 0xFFFFFF) - 1] = entry.ref
        else:
            toc_entries = Array(toc_header.count, toc_entry_struct).parse(data[offset:offset+(toc_header.count*5)])
            self.entries = [e.ref for e in toc_entries]

        self.primary = toc_header.primary & 0x7FFFFFFF

    def num_entries(self):
        return len(self.entries)

    def get_offset(self, handle):
        return self.entries[handle - 1]

In [59]:
toc = StoreToc()
toc.parse(data, store_header.iRef)
for i in range(toc.num_entries()):
    print(f"{toc.get_offset(i + 1):08x}")

0001cdd1
00000000
00007fdb
0001c8bd
0001c8cc
8300000e
80000013
0001b165
80000017
0001bf1d
85000007
0001c39d
80000014
8100000d
00000000
0001c8ec
80000015
80000019
82000006
82000011
80000012
0001bd04
8200000f
000082c6
8c000009


In [60]:
db_schema_offset = toc.get_offset(toc.primary)
db_schema = db_schema_struct.parse(data[db_schema_offset:])

print(db_schema)

Container: 
    uid = 268435561
    iVersion = 3
    iToken = 1
    tables = ListContainer: 
        Container: 
            name = u'iApplis' (total 7)
            columns = ListContainer: 
                Container: 
                    name = u'app_No' (total 6)
                    type = 3
                    attributes = 0
                    maxLength = None
                Container: 
                    name = u'appType' (total 7)
                    type = 1
                    attributes = 0
                    maxLength = None
                Container: 
                    name = u'folderNo' (total 8)
                    type = 3
                    attributes = 0
                    maxLength = None
                Container: 
                    name = u'appName' (total 7)
                    type = 11
                    attributes = 0
                    maxLength = 50
                Container: 
                    name = u'appNameUni' (total 10)
                    ty

In [68]:
for table in db_schema.tables:
    # convert the schema to Construct schema
    columns_schemas = [
        "_rowSize" / TCardinality
    ]
    for column in table.columns:
        column_schema = None
        if column.type == 0:
            column_schema = ReadBitSequence()
        elif column.type == 1:
            column_schema = Int8sl
        elif column.type == 2:
            column_schema = Int8ul
        elif column.type == 3:
            column_schema = Int16sl
        elif column.type == 4:
            column_schema = Int16ul
        elif column.type == 5:
            column_schema = Int32sl
        elif column.type == 6:
            column_schema = Int32ul
        elif column.type == 7:
            column_schema = Int64sl
        elif column.type == 8:
            column_schema = Float32l
        elif column.type == 9:
            column_schema = Float64l
        elif column.type == 10:
            column_schema = Int64sl # datetime
        elif column.type == 11:
            column_schema = PascalString(Int8ul, "cp932")
        elif column.type == 12:
            # Symbian uses SCSU for unicode strings
            column_schema = PascalString(TCardinality, "SCSU")
        elif column.type == 13:
            column_schema = Prefixed(Int8ul, GreedyBytes())
        elif column.type == 14 or column.type == 15 or column.type == 16:
            data_schema = None
            match column.type:
                case 14:
                    data_schema = PascalString(Int8ul, "cp932")
                case 15:
                    data_schema = PascalString(TCardinality, "SCSU")
                case 16:
                    data_schema = Prefixed(Int8ul, GreedyBytes())
            column_schema = Struct(
                "isInline" / ReadBitSequence(),
                "outOfLineData" / If(not this.isInline, Struct(
                    "packedBlobId" / TCardinality,
                    "size" / TCardinality
                )),
                "data" / If(this.isInline, data_schema)
            )            
        else:
            raise ValueError(f"column type {column.type} not supported")
        
        if (column.attributes & 1) == 0:
            column_schema = FocusedSeq(
                "data",
                "exists" / ReadBitSequence(),
                "data" / If(this.exists, column_schema)
            )
        
        # we use Optional because if there are no following entries, eof can be premature
        columns_schemas.append(column.name / Optional(column_schema))

    table_construct_schema = Struct(*columns_schemas)

    table_token_offset = toc.get_offset(table.iTokenId)
    print(toc.entries)
    table_token = table_token_struct.parse(data[table_token_offset:])
    
    cur_cluster_id = table_token.iHead
    
    want = ["appName","appVersion","packageUrl","profileVersion","jar_Size","spSize0","spSize1","spSize2","spSize3","spSize4","spSize5","spSize6","spSize7","spSize8","spSize9","spSize10","spSize11","spSize12","spSize13","spSize14","spSize15","appClass","appParam","lastModifiedTime","useNetwork","targetDevice","launchAt","myConcierge","useTelephone","useBrowser","allowLaunchUrl","allowLaunchMail","allowPushBy","launchUrlValid","launchMailValid","pushByValid","appTrace","drawAreaWidth","drawAreaHeight","getSysInfo","getTerminalId","getUserId","getUtnValid","utnDownload","jamFileName","jarFileName","iccid","messageCode","entryId","trustedApid","launchApp","launchByDeny","accessUserInfo","getPrivateInfo"]
    
    while cur_cluster_id != 0:
        cluster_offset = toc.get_offset(cur_cluster_id)
        cluster = cluster_struct.parse(data[cluster_offset:])
        try:
            for record_data in cluster.data:
                if record_data is None: continue
                print("-"*10)
                bigwant = table_construct_schema.parse(record_data)
                if bigwant["appName"] == None: continue
                for wantit in want:
                    print(wantit, ": ", bigwant[wantit])
        except StreamError:
            # ignore incomplete entries
            pass

        cur_cluster_id = cluster.iNext & 0xFFFFFF

[118225, 0, 32731, 116925, 116940, 2197815310, 2147483667, 110949, 2147483671, 114461, 2231369735, 115613, 2147483668, 2164260877, 0, 116972, 2147483669, 2147483673, 2181038086, 2181038097, 2147483666, 113924, 2181038095, 33478, 2348810249]
----------
----------
appName :  電子ﾏﾈｰ「Edy」
appVersion :  20050315a
packageUrl :  Container: 
    isInline = 1
    outOfLineData = None
    data = u'http://mobile.bitwallet.co.jp/dw'... (truncated, total 66)
profileVersion :  DoJa-4.0
jar_Size :  42165
spSize0 :  -1
spSize1 :  -1
spSize2 :  -1
spSize3 :  -1
spSize4 :  -1
spSize5 :  -1
spSize6 :  -1
spSize7 :  -1
spSize8 :  -1
spSize9 :  -1
spSize10 :  -1
spSize11 :  -1
spSize12 :  -1
spSize13 :  -1
spSize14 :  -1
spSize15 :  -1
appClass :  Container: 
    isInline = 1
    outOfLineData = None
    data = u'EdyOn' (total 5)
appParam :  None
lastModifiedTime :  63280241972000000
useNetwork :  1
targetDevice :  F901iC
launchAt :  None
myConcierge :  0
useTelephone :  0
useBrowser :  1
allowLaunchUrl :  